In [63]:
import pandas as pd
# read_csv方法中有3个参数，第1个参数是加载文本文件的路径，第2个关键字参数sep是分隔符，第3个关键字参数header是文本文件的第1行是否为字段名
train_df = pd.read_csv('data/text_train.txt', sep='\t', header=None)
train_df.columns = ['分类', '内容']
train_df.head() # 打印训练集前5行

,分类,内容
0,娱乐,《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...
1,娱乐,６．１６日剧榜 ＜最后的朋友＞ 亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...
2,娱乐,超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...
3,娱乐,吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...
4,娱乐,组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...


In [64]:
# 查看训练集每个分类的名字以及样本数量
for name, group in train_df.groupby(train_df.columns[0]):
    print(name,len(group))


体育 2000
健康 2000
女人 2000
娱乐 2000
房地产 2000
教育 2000
文化 2000
新闻 2000
旅游 2000
汽车 2000
科技 2000
财经 2000


In [65]:
train_df.groupby('分类').count().reset_index()

,分类,内容
0,体育,2000
1,健康,2000
2,女人,2000
3,娱乐,2000
4,房地产,2000
5,教育,2000
6,文化,2000
7,新闻,2000
8,旅游,2000
9,汽车,2000


In [66]:
# 文本分类不是任何词都是重要的判断语句，针对一些没有任何代表意义的词（的、了、么、吗）应该去掉，这种词容易对模型进行干扰
with open('data/stopwords.txt', encoding='utf8') as file:
    line_list = file.readlines()
    stopword_list = [k.strip() for k in line_list]
    stopword_set = set(stopword_list)
    print('停顿词列表，即变量stopword_list中共有%d个元素' %len(stopword_list))
    print('停顿词集合，即变量stopword_set中共有%d个元素' %len(stopword_set))

停顿词列表，即变量stopword_list中共有1452个元素
停顿词集合，即变量stopword_set中共有1213个元素


In [67]:
import jieba
import time

In [68]:
def word_cut(content, stopword_set):
    cutWords = [k for k in jieba.cut(content, True) if k not in stopword_set]
    return " ".join(cutWords)

In [69]:
# 训练样本内容分词
train_df["内容"] = train_df["内容"].map(lambda x: word_cut(x, stopword_set))
train_df.head()

,分类,内容
0,娱乐,青蛇 造型 造型师 默认 新 红楼 红楼梦 额 妆 抄袭 图 看过 过电 电影 青蛇 不...
1,娱乐,６．１６ 日剧 榜 ＜ 最后 朋友 ＞ 亮 最后 杀招 成功 登顶 最后 朋友 ...
2,娱乐,超乎 想象 好看 纳尼亚 尼亚 传奇 ２： 凯斯 宾 王子 现时 资讯 发达 搜狐 电影...
3,娱乐,吴宇森 赤壁 赤壁大战 大战 不会 出现 现在 上集 希望 赤壁 不一 感觉 。” 刚刚...
4,娱乐,组 图 ：《 多情 女人 痴情 男 陈浩民 现场 耍宝 陈浩民 外面 朋友 现在 搜狐 ...


In [70]:
# 测试样本内容分词
test_df = pd.read_csv('data/text_test.txt', sep='\t', header=None)
test_df.columns = ['分类', '内容']
test_df["内容"] = test_df["内容"].map(lambda x: word_cut(x, stopword_set))
test_df.head()

,分类,内容
0,娱乐,组 图 黄健翔 拍 时装 大片 承认 口 遮拦 ２００６ 年 之前 最好 体育 体育...
1,娱乐,奥运 明星 明星写真 写真 写真集 集锦 曝光 展现 健康 时尚 图 来源 人民 人...
2,娱乐,内地 票房 票房榜 ：《 功夫 熊猫 获 全胜 带动 内地 影 市 功夫 熊猫 首映...
3,娱乐,编者 编者按 昨天 央视 紧急 停播 动画 动画片 画片 虹 猫 蓝 兔 七 侠 传 事...
4,娱乐,第十 第十一 第十一届 十一 十一届 一届 上海 海国 国际 电影 电影节 金 爵 奖评...


In [71]:
# label映射
label_map ={}
for name, group in train_df.groupby(train_df.columns[0]):
    print(name, len(group))
    label_map[name] = "__label__" + str(name)
label_map

体育 2000
健康 2000
女人 2000
娱乐 2000
房地产 2000
教育 2000
文化 2000
新闻 2000
旅游 2000
汽车 2000
科技 2000
财经 2000


{'体育': '__label__体育',
 '健康': '__label__健康',
 '女人': '__label__女人',
 '娱乐': '__label__娱乐',
 '房地产': '__label__房地产',
 '教育': '__label__教育',
 '文化': '__label__文化',
 '新闻': '__label__新闻',
 '旅游': '__label__旅游',
 '汽车': '__label__汽车',
 '科技': '__label__科技',
 '财经': '__label__财经'}

In [72]:
train_df["分类"] = train_df["分类"].map(lambda x: label_map[x])
train_df

,分类,内容
0,__label__娱乐,青蛇 造型 造型师 默认 新 红楼 红楼梦 额 妆 抄袭 图 看过 过电 电影 青蛇 不...
1,__label__娱乐,６．１６ 日剧 榜 ＜ 最后 朋友 ＞ 亮 最后 杀招 成功 登顶 最后 朋友 ...
2,__label__娱乐,超乎 想象 好看 纳尼亚 尼亚 传奇 ２： 凯斯 宾 王子 现时 资讯 发达 搜狐 电影...
3,__label__娱乐,吴宇森 赤壁 赤壁大战 大战 不会 出现 现在 上集 希望 赤壁 不一 感觉 。” 刚刚...
4,__label__娱乐,组 图 ：《 多情 女人 痴情 男 陈浩民 现场 耍宝 陈浩民 外面 朋友 现在 搜狐 ...
...,...,...
23995,__label__女人,鉴别 穿衣 男人 泡妞 高手 男人 什么 吸引 女人 真实 深刻 胆识 幽默 气度 浪漫...
23996,__label__女人,每期 节目 搜狐 女人 婚嫁 频道 回放 保留 留美 美好 回忆 １． 北京 北京电台 ...
23997,__label__女人,风波 风波不断 不断 ＬＶ 受 谣言 言中 中国 中国区 首度 停业 ＬＶ 最近 有点 ...
23998,__label__女人,夏日 最好 苦瓜 减肥 减肥法 疯狂 瘦身 苦瓜 减肥 风 好象 吹 很久 确实 苦瓜 ...


In [73]:
# 测试集，标签映射
test_df["分类"] = test_df["分类"].map(lambda x: label_map[x])
test_df

,分类,内容
0,__label__娱乐,组 图 黄健翔 拍 时装 大片 承认 口 遮拦 ２００６ 年 之前 最好 体育 体育...
1,__label__娱乐,奥运 明星 明星写真 写真 写真集 集锦 曝光 展现 健康 时尚 图 来源 人民 人...
2,__label__娱乐,内地 票房 票房榜 ：《 功夫 熊猫 获 全胜 带动 内地 影 市 功夫 熊猫 首映...
3,__label__娱乐,编者 编者按 昨天 央视 紧急 停播 动画 动画片 画片 虹 猫 蓝 兔 七 侠 传 事...
4,__label__娱乐,第十 第十一 第十一届 十一 十一届 一届 上海 海国 国际 电影 电影节 金 爵 奖评...
...,...,...
11995,__label__女人,撞破 男友 旧情 情人 同床 同床共枕 口述 何方 独 异乡 寂寞 日子 怕 无法 遏止...
11996,__label__女人,画 诠释 毕加索 人生 图 第二 第二部 二部 部分 凄冷 蓝色 １９０１－１９０４...
11997,__label__女人,女人 欧洲 欧洲杯 四大 大理 理由 四年 一度 欧洲 欧洲杯 进入 现在 进行 行时 ...
11998,__label__女人,女人 暴露 既要 性感 不失 失去 品味 千呼万唤 始 犹抱琵琶半遮面 琵琶 琵琶半...


In [74]:
train_df.to_csv("data/ft_train_df2.txt", sep="\t", index=False, header=None)
test_df.to_csv("data/ft_test_df2.txt", sep="\t", index=False, header=None)

In [75]:
import fasttext
classifier = fasttext.train_supervised(input='data/ft_train_df2.txt', dim=100, epoch=10,
                                         lr=0.1, wordNgrams=2, loss='softmax', label="__label__")

Read 29M words
Number of words:  354901
Number of labels: 12
Progress: 100.0% words/sec/thread: 2388528 lr:  0.000000 avg.loss:  1.446353 ETA:   0h 0m 0s


In [76]:
result = classifier.test('data/ft_test_df2.txt')
print(result)

(12000, 0.78375, 0.78375)


In [115]:
z = labelEncoder.fit_transform(train_df['分类'])

In [116]:
test_label_list = labelEncoder.transform(test_df['分类'])
test_label_list

array([3, 3, 3, ..., 2, 2, 2])

In [122]:
import numpy as np
test_df_ = np.array(test_df["内容"]).tolist()
predict_label_list = [y for x in test_df_ for y in classifier.predict(x)[0]]
predict_label_list[:5]

['__label__娱乐', '__label__体育', '__label__娱乐', '__label__女人', '__label__娱乐']

In [119]:
predict_label_list = labelEncoder.transform(predict_label_list)
predict_label_list

array([3, 0, 3, ..., 2, 2, 3])

In [104]:
labelEncoder.classes_=[x.strip('__label__') for x in labelEncoder.classes_]
labelEncoder.classes_

['体育', '健康', '女人', '娱乐', '房地产', '教育', '文化', '新闻', '旅游', '汽车', '科技', '财经']

In [105]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

,体育,健康,女人,娱乐,房地产,教育,文化,新闻,旅游,汽车,科技,财经
体育,964,1,5,13,0,3,4,1,2,0,3,4
健康,0,832,42,1,0,20,15,43,4,0,20,23
女人,8,47,743,86,0,24,61,4,11,8,8,0
娱乐,6,0,48,713,0,9,215,1,5,0,3,0
房地产,1,0,8,1,877,3,0,13,10,4,3,80
教育,1,15,28,5,2,888,13,26,2,1,8,11
文化,4,13,32,118,3,9,752,22,23,5,17,2
新闻,11,48,24,7,13,76,101,488,23,6,56,147
旅游,2,13,49,34,20,4,61,23,757,4,21,12
汽车,15,5,3,7,2,23,1,6,5,882,13,38


In [113]:
from sklearn.metrics import precision_recall_fscore_support
def ft_eval_model(test_label_list, predict_label_list, className_list):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(test_label_list, predict_label_list)
    # 计算总体的平均Precision, Recall, f1, support
    total_p = np.average(p, weights=s)
    total_r = np.average(r, weights=s)
    total_f1 = np.average(f1, weights=s)
    total_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': className_list,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [total_p],
        u'Recall': [total_r],
        u'F1': [total_f1],
        u'Support': [total_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

ft_eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.950690,0.96400,0.957299,1000
1,健康,0.824579,0.83200,0.828273,1000
2,女人,0.753550,0.74300,0.748238,1000
3,娱乐,0.714429,0.71300,0.713714,1000
4,房地产,0.905057,0.87700,0.890808,1000
5,教育,0.817680,0.88800,0.851390,1000
6,文化,0.593997,0.75200,0.663725,1000
7,新闻,0.648074,0.48800,0.556760,1000
8,旅游,0.874134,0.75700,0.811361,1000
9,汽车,0.944325,0.88200,0.912099,1000
